In [3]:
# Land Cover Data for MCDA from Google Earth Engine
import ee
import json

# Authenticate and initialize Earth Engine
try:
    ee.Initialize()
except Exception:
    ee.Authenticate()
    ee.Initialize()

# Load ESA WorldCover 10m land cover dataset (2020)
landcover = ee.Image("ESA/WorldCover/v100/2020")

# Load simplified country boundaries
countries = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017")

# Get geometries for Denmark and Norway
denmark = countries.filter(ee.Filter.eq("country_na", "Denmark"))
norway = countries.filter(ee.Filter.eq("country_na", "Norway"))

# Add a "country_code" property: Denmark = 2, Norway = 1
denmark = denmark.map(lambda f: f.set("country_code", 2))
norway = norway.map(lambda f: f.set("country_code", 1))

# Combine the country features
combined_countries = denmark.merge(norway)

# Rasterize the country features
country_raster = ee.Image().int().paint(combined_countries, "country_code")

# Stack landcover and country raster
combined_image = landcover.addBands(country_raster.rename("country"))

# Sample both landcover and country in a single operation
samples = combined_image.sample(
    region=combined_countries.geometry(),
    scale=500,
    numPixels=5000,
    geometries=True
)

# Get the data
features = samples.getInfo()['features']

# Land cover class names
legend = {
    10: "Tree Cover",
    20: "Shrubland",
    30: "Grassland",
    40: "Cropland",
    50: "Built-up",
    60: "Bare / Sparse",
    70: "Snow and Ice",
    80: "Water",
    90: "Wetlands",
    95: "Mangroves",
    100: "Moss & Lichen"
}

# Country code to name
country_map = {1: "Norway", 2: "Denmark"}

# Land Cover Outfile File
output_dir = r"C:\GISDataManipulation\LandCoverData"
os.makedirs(output_dir, exist_ok=True)
output_file = os.path.join(output_dir, "landcover_with_country.txt")

# Save the data
with open(output_file, "w") as f:
    f.write("Latitude, Longitude, Country, Class Value, Class Name\n")
    for feat in features:
        lon, lat = feat["geometry"]["coordinates"]
        land_val = feat["properties"]["Map"]
        country_val = feat["properties"]["country"]
        label = legend.get(land_val, "Unknown")
        country = country_map.get(country_val, "Unknown")
        f.write(f"{lat:.6f}, {lon:.6f}, {country}, {land_val}, {label}\n")

print(f"Land Cover Data saved at: {output_file}")



Successfully saved authorization token.


NameError: name 'os' is not defined

In [4]:
#Substation Data Fetching for Norway and Denmark
import requests
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

# Output paths
OUTPUT_GEOJSON = r"C:\GISDataManipulation\SubstationData\Substations_Norway_Denmark_Recent.geojson"
OUTPUT_TXT = r"C:\GISDataManipulation\SubstationData\Substations_Norway_Denmark_Recent.txt"

# Bounding box for Norway and Denmark: (south, west, north, east)
bbox = [54.5, 5.0, 71.0, 31.0]

# Overpass API query for substations
query = f"""
[out:json][timeout:300];
(
  node["power"="substation"]({bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]});
  way["power"="substation"]({bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]});
);
out body;
>;
out skel qt;
"""

# Request from Overpass API
print("Fetching substations from OSM (Overpass API)...")
url = "https://overpass-api.de/api/interpreter"
response = requests.post(url, data={'data': query})
response.raise_for_status()
data = response.json()

# Parse Overpass data to GeoDataFrame
elements = data['elements']
nodes = {el['id']: (el['lon'], el['lat']) for el in elements if el['type'] == 'node'}
features = []

for el in elements:
    if el['type'] == 'node':
        geometry = Point(el['lon'], el['lat'])
        features.append({'geometry': geometry, 'id': el['id'], 'type': 'node'})
    elif el['type'] == 'way' and 'nodes' in el:
        coords = [nodes[nid] for nid in el['nodes'] if nid in nodes]
        if coords:
            centroid = Point(pd.DataFrame(coords).mean().values)
            features.append({'geometry': centroid, 'id': el['id'], 'type': 'way'})

gdf = gpd.GeoDataFrame(features, crs="EPSG:4326")

# Save GeoJSON
gdf.to_file(OUTPUT_GEOJSON, driver="GeoJSON")
print(f" Substation data saved to: {OUTPUT_GEOJSON}")

# Save simplified .txt file with centroids
gdf['Longitude'] = gdf.geometry.x
gdf['Latitude'] = gdf.geometry.y
df_txt = gdf[['Latitude', 'Longitude', 'id', 'type']]
df_txt.to_csv(OUTPUT_TXT, sep=",", index=False)
print(f" Substation locations saved to TXT at: {OUTPUT_TXT}")

# Preview output
print("\n Sample of fetched substation data:")
print(df_txt.head())

Fetching substations from OSM (Overpass API)...
 Substation data saved to: C:\GISDataManipulation\SubstationData\Substations_Norway_Denmark_Recent.geojson
 Substation locations saved to TXT at: C:\GISDataManipulation\SubstationData\Substations_Norway_Denmark_Recent.txt

 Sample of fetched substation data:
    Latitude  Longitude         id  type
0  60.647636  17.146934  124440614  node
1  63.406501  10.481262  245431610  node
2  63.385097  10.357548  246519073  node
3  63.412951  10.539669  247455681  node
4  63.416624  10.535464  247455692  node


In [ ]:
#Transmission Line Data Fetching for Norway and Denmark
import requests
import geopandas as gpd
import pandas as pd
from shapely.geometry import LineString

# Output paths
OUTPUT_TRANSMISSION_GEOJSON = r"C:\GISDataManipulation\TransmissionLineData\Transmission_Lines_Norway_Denmark_Recent.geojson"
OUTPUT_TRANSMISSION_TXT = r"C:\GISDataManipulation\TransmissionLineData\Transmission_Lines_Norway_Denmark_Recent.txt"

# Bounding box for Denmark and Norway: (south, west, north, east)
bbox = [54.5, 5.0, 71.0, 31.0]

# Overpass query for power lines
query = f"""
[out:json][timeout:300];
(
  way["power"="line"]({bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]});
);
out body;
>;
out skel qt;
"""

# Fetch data
url = "https://overpass-api.de/api/interpreter"
response = requests.post(url, data={'data': query})
response.raise_for_status()
data = response.json()

# Convert JSON to GeoDataFrame
nodes = {node['id']: (node['lon'], node['lat']) for node in data['elements'] if node['type'] == 'node'}
lines = []
for element in data['elements']:
    if element['type'] == 'way' and 'nodes' in element:
        coords = [nodes[node_id] for node_id in element['nodes'] if node_id in nodes]
        lines.append({'geometry': LineString(coords), 'id': element['id']})

gdf = gpd.GeoDataFrame(lines, crs="EPSG:4326")

# Show first few lines
print("First 5 fetched transmission lines:")
print(gdf.head(5))

# Save GeoJSON
gdf.to_file(OUTPUT_TRANSMISSION_GEOJSON, driver="GeoJSON")
print(f"Transmission lines saved to {OUTPUT_TRANSMISSION_GEOJSON}")

# Convert centroids and save to TXT
gdf['Longitude'] = gdf.geometry.centroid.x
gdf['Latitude'] = gdf.geometry.centroid.y
df_txt = gdf[['Latitude', 'Longitude', 'id']]
df_txt.to_csv(OUTPUT_TRANSMISSION_TXT, sep=",", index=False)
print(f"Transmission line centroids saved to TXT at: {OUTPUT_TRANSMISSION_TXT}")


In [5]:
# Transmission and Substation File Merging
import pandas as pd

# Load the two input files
trans = pd.read_csv(r"C:\GISDataManipulation\TransmissionLineData\Transmission_Lines_Norway_Denmark_Recent.txt")
subs = pd.read_csv(r"C:\GISDataManipulation\SubstationData\Substations_Norway_Denmark_Recent.txt")

# Add type columns
trans["type"] = "Transmission Line"
subs["type"] = "Substation"

# Keep only necessary columns
trans = trans[["Latitude", "Longitude", "id", "type"]]
subs = subs[["Latitude", "Longitude", "id", "type"]]

# Merge both DataFrames
combined = pd.concat([trans, subs], ignore_index=True)

# Save to new .txt file
combined.to_csv(r"C:\GISDataManipulation\StationTranmissionLineCombined\merged_grid_infra.txt", index=False)

print("Merged infrastructure saved to 'merged_grid_infra.txt'")


Merged infrastructure saved to 'merged_grid_infra.txt'
